In [1]:
import os
import json
import requests

In [2]:
api_key = os.getenv('SERPAPI_API_KEY')

params = {
    "api_key": api_key,
    "engine": "yelp_reviews",
    "place_id": "wEYj3B_EH6JS6DbfmZQudw", # example: Twenty Seven Steps, Christchurch, New Zealand
    "sortby": "date_desc"
}

In [3]:
search = requests.get("https://serpapi.com/search", params=params)
data = search.json()

In [4]:
# Pretty-print first 2 reviews for checking
for review in data.get("reviews", [])[:2]:
    user = review.get("user", {}).get("name")
    rating = review.get("rating")
    date = review.get("date")
    text = review.get("text") or ""
    print(f"{user} rated {rating}⭐ on {date}")
    print(text[:200])
    print("-" * 60)

John B. rated 5⭐ on 2025-03-27T05:04:34Z

------------------------------------------------------------
Azure G. rated 5⭐ on 2025-02-14T04:09:56Z

------------------------------------------------------------


In [5]:
print(json.dumps(data, indent=2, ensure_ascii=False))

{
  "search_metadata": {
    "id": "69127836556ce392e0941c60",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/9caa040754988d63/69127836556ce392e0941c60.json",
    "created_at": "2025-11-10 23:41:42 UTC",
    "processed_at": "2025-11-10 23:41:42 UTC",
    "yelp_reviews_url": "https://www.yelp.com/biz/wEYj3B_EH6JS6DbfmZQudw?sort_by=date_desc",
    "raw_html_file": "https://serpapi.com/searches/9caa040754988d63/69127836556ce392e0941c60.html",
    "prettify_html_file": "https://serpapi.com/searches/9caa040754988d63/69127836556ce392e0941c60.prettify",
    "total_time_taken": 4.72
  },
  "search_parameters": {
    "engine": "yelp_reviews",
    "start": 0,
    "num": "49",
    "sortby": "date_desc",
    "place_id": "wEYj3B_EH6JS6DbfmZQudw"
  },
  "search_information": {
    "total_results": 46,
    "business": "Twenty Seven Steps"
  },
  "review_languages": [
    {
      "hl": "en",
      "count": 46
    },
    {
      "hl": "fr",
      "count": 1
    },
    {
   

### READ THE RESTAURANT PLACE IDS ACROSS CHC

In [15]:
import pandas as pd 

In [16]:
restaurant_id = pd.read_csv("Christchurch_place_ids.csv")

restaurant_id.head()

,place_id,title,rating,reviews,price,categories,link
0,wEYj3B_EH6JS6DbfmZQudw,Twenty Seven Steps,4.8,48,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
1,twenty-seven-steps-christchurch,Twenty Seven Steps,4.8,48,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
2,5ZOai8-HCajxJ4gnjqsAdw,OGB,4.9,9,NaN,"Bars, New Zealand",https://www.yelp.com/biz/ogb-christchurch?osq=...
3,ogb-christchurch,OGB,4.9,9,NaN,"Bars, New Zealand",https://www.yelp.com/biz/ogb-christchurch?osq=...
4,iL_451H-5M9tmaAsFqKZrw,Orange House,5.0,2,NaN,Chinese,https://www.yelp.com/biz/orange-house-christch...


In [44]:
# 1️⃣ Find titles that appear more than once
dup_titles = (
    restaurant_id.groupby("title")
    .size()
    .reset_index(name="count")
    .query("count > 1")
    .sort_values("count", ascending=False)
)

print(f"🔁 Total duplicated restaurant titles: {len(dup_titles)}")

# 2️⃣ Display all rows (place_id + title) for duplicated titles
duplicate_details = (
    restaurant_id[restaurant_id["title"].isin(dup_titles["title"])]
    .loc[:, ["place_id", "title", "rating", "reviews", "price", "categories"]]
    .sort_values(["title", "place_id"])
    .reset_index(drop=True)
)

print("Sample of duplicated restaurants:")
duplicate_details.head(20)

🔁 Total duplicated restaurant titles: 186
Sample of duplicated restaurants:


,place_id,title,rating,reviews,price,categories
0,5th-street-christchurch,5th Street,5.0,6,NaN,"Wine Bars, Tapas/Small Plates, Cocktail Bars"
1,Busy8uaGSr4xVTDK64KCcg,5th Street,5.0,6,NaN,"Wine Bars, Tapas/Small Plates, Cocktail Bars"
2,Bonsa2BwNhdqFWVmrn1Rag,Abalone Thai Restaurant,4.0,4,NaN,Thai
3,abalone-thai-restaurant-christchurch-city,Abalone Thai Restaurant,4.0,4,NaN,Thai
4,5aSFsf5QbevEYxH3KTkRKA,Amaterrace Teppanyaki,4.0,2,NaN,Teppanyaki
5,amaterrace-teppanyaki-christchurch,Amaterrace Teppanyaki,4.0,2,NaN,Teppanyaki
6,amazonita-christchurch,Amazonita,4.1,10,NaN,Mediterranean
7,zHk2a3tQdyldANFOeqjR1A,Amazonita,4.1,10,NaN,Mediterranean
8,FfB040u_JQh6PnbsrLQM_w,Anna’s Cafe,4.0,1,NaN,"Breakfast & Brunch, Cafes"
9,annas-cafe-christchurch,Anna’s Cafe,4.0,1,NaN,"Breakfast & Brunch, Cafes"


In [45]:
print("\n📋 List of all duplicated restaurant titles:\n")
for t in dup_titles["title"].tolist():
    print("-", t)


📋 List of all duplicated restaurant titles:

- Hachi Hachi
- Spagalimis
- McDonald’s
- Fox & Ferret Public Ale House
- Riccarton Noodle House
- Pot Sticker Dumpling Bar
- Prince of Persia
- Pure Cafe Co.
- Rangoon Ruby
- Raw Sugar Cafe
- Reality Bites
- Red Elephant
- Red Rock Cafe
- Roast Ease
- Pies and Coffee
- Robert Harris
- Rogues of Rotherham
- Rose Cafe
- Saigon Star
- Sampan House Restaurant
- Samurai Bowl
- Sevanti Rest & Bar
- Silver Dolphin
- Portershed
- 5th Street
- Slick Burgers
- Pescatore
- Mitchelli’s Cafe Rinato
- Monster Chicken
- Morrell & Co
- Mosaic by Simo
- Mr Sushi
- Mumbaiwala
- Mum’s
- Nobanno Restaurant Bar & Cafe
- NomNom Kitchen
- Noodle
- Noodle Union
- North and South Gourmet
- OGB
- Orange House
- Original Sin
- Orleans
- Papas Pizza
- Passengers & Co
- Pedro’s House of Lamb
- Pier Side Cafe
- Smoke ‘n’ Barrel
- Mexico Christchurch
- Tutto Bene
- The Rockpool
- The Twisted Hop
- The Watershed
- Tiki Taco
- Tony’s Teppan Yaki Japanese Restaurant
- Town

In [23]:
restaurant_id.columns

Index(['place_id', 'title', 'rating', 'reviews', 'price', 'categories',
       'link'],
      dtype='object')

In [18]:
restaurant_id.isnull().sum()

place_id        0
title           0
rating          0
reviews         0
price         252
categories      0
link            0
dtype: int64

In [19]:
restaurant_id["place_id"].duplicated().sum()

np.int64(0)

### TEST SCRAPPED REVIEWS BASED ON RESTAURANT PLACE ID 

In [24]:
import os
import json
import csv
import time
import requests
import pandas as pd

In [26]:
API_KEY = os.getenv("SERPAPI_API_KEY")  # make sure you ran: export SERPAPI_API_KEY="your_key"
BASE_URL = "https://serpapi.com/search"
INPUT_CSV = "data/Christchurch_place_ids.csv"  # the file from Stage 1
REVIEWS_PER_PAGE = 49

In [27]:
# Load place_ids csv 
df = pd.read_csv(INPUT_CSV)
first_place_id = df["place_id"].iloc[0]   # just test with the first one
print(f"Testing SerpApi review scrape for place_id: {first_place_id}")

Testing SerpApi review scrape for place_id: wEYj3B_EH6JS6DbfmZQudw


In [28]:
# --- Step 2: Build request parameters ---
params = {
    "api_key": API_KEY,
    "engine": "yelp_reviews",
    "place_id": first_place_id,
    "num": REVIEWS_PER_PAGE,
    "start": 0,
    "sortby": "date_desc"   # newest first
}

In [29]:
# --- Step 3: Send request to SerpApi ---
response = requests.get(BASE_URL, params=params, timeout=30)
response.raise_for_status()
data = response.json()

In [30]:
# --- Step 4: Inspect response structure ---
print("Top-level keys:", list(data.keys()))
print("Number of reviews fetched:", len(data.get("reviews", [])))

Top-level keys: ['search_metadata', 'search_parameters', 'search_information', 'review_languages', 'reviews']
Number of reviews fetched: 46


In [33]:
# --- Step 5: Print a few sample reviews ---
for i, review in enumerate(data.get("reviews", [])[:5], start=1):
    user = review.get("user", {}).get("name")
    rating = review.get("rating")
    date = review.get("date")
    text = review.get("comment", {}).get("text", "")
    print(f"Review #{i}")
    print(f"User: {user}")
    print(f"Rating: {rating}")
    print(f"Date: {date}")
    print(f"Text: {text[:200]}...")

Review #1
User: John B.
Rating: 5
Date: 2025-03-27T05:04:34Z
Text: A warm greeting followed by a delicious dinner served by friendly happy servers .... What more can you want ?? I had read about their duck parfait app which to my dismay was not on the menu that night...
Review #2
User: Azure G.
Rating: 5
Date: 2025-02-14T04:09:56Z
Text: Twenty Seven Steps - what a spot! Intimate space in a lovely location with great service, expansive wine selection and they are happy to make you a tasty mocktail that suits your palate. 

Now... the ...
Review #3
User: Calista C.
Rating: 5
Date: 2025-01-29T21:31:56Z
Text: Visited here while on my stop in Christchurch. Easily the best meal I had in town. They're on the second floor, above their sister restaurant, Downstairs. Upon being seated, the server was extremely t...
Review #4
User: Jasper R.
Rating: 5
Date: 2024-11-20T04:47:31Z
Text: Everything in here was amazing! One of my favorite places!

The service is top-notch. They are so attentive and fl

### READ THE RESTAURANT REVIEW IN CHC

In [34]:
restaurant_reviews = pd.read_csv('christchurch_reviews_all_pages.csv')

restaurant_reviews.head()

,place_id,review_type,user,rating,date,text,review_position,user_address,useful,cool,funny,user_link
0,wEYj3B_EH6JS6DbfmZQudw,recommended,John B.,5,2025-03-27T05:04:34Z,A warm greeting followed by a delicious dinner...,1,"Falmouth, MA",0,0,0,https://www.yelp.com/user_details?userid=MgHxF...
1,wEYj3B_EH6JS6DbfmZQudw,recommended,Azure G.,5,2025-02-14T04:09:56Z,Twenty Seven Steps - what a spot! Intimate spa...,2,"Denver, CO",0,0,0,https://www.yelp.com/user_details?userid=Z583n...
2,wEYj3B_EH6JS6DbfmZQudw,recommended,Calista C.,5,2025-01-29T21:31:56Z,Visited here while on my stop in Christchurch....,3,"Irvine, CA",2,2,0,https://www.yelp.com/user_details?userid=PqR0K...
3,wEYj3B_EH6JS6DbfmZQudw,recommended,Jasper R.,5,2024-11-20T04:47:31Z,Everything in here was amazing! One of my favo...,4,"Lathrop, CA",0,0,0,https://www.yelp.com/user_details?userid=IaX6k...
4,wEYj3B_EH6JS6DbfmZQudw,recommended,Andy M.,4,2024-10-25T05:19:39Z,"Cool atmosphere, great service, and the good f...",5,"Cincinnati, OH",0,0,0,https://www.yelp.com/user_details?userid=ODxAj...


In [52]:
# Load reviews
reviews = pd.read_csv("data/christchurch_reviews_all_pages.csv")

# Example duplicate restaurant
duplicates = {
    "Twenty Seven Steps": ["wEYj3B_EH6JS6DbfmZQudw", "twenty-seven-steps-christchurch"]
}

for title, ids in duplicates.items():
    subset = reviews[reviews["place_id"].isin(ids)]
    print(f"=== {title} ===")
    print(f"Total reviews scraped: {len(subset)}")
    print(f"Unique reviews (by user + date + text): {len(subset.drop_duplicates(subset=['user','date','text']))}")
    print()

=== Twenty Seven Steps ===
Total reviews scraped: 46
Unique reviews (by user + date + text): 46



In [53]:
# Load datasets
places = pd.read_csv("data/Christchurch_place_ids.csv")[["place_id", "title"]]
reviews = pd.read_csv("data/christchurch_reviews_all_pages.csv")[["place_id", "user", "date", "text", "rating"]]

# Normalize restaurant titles (case-insensitive, strip spaces)
places["title_lower"] = places["title"].str.lower().str.strip()

# Find restaurants with multiple place_ids
duplicate_titles = (
    places.groupby("title_lower")
    .filter(lambda x: len(x["place_id"].unique()) > 1)
)

# Create a list of results
results = []

for title, group in duplicate_titles.groupby("title_lower"):
    place_ids = group["place_id"].tolist()
    title_original = group["title"].iloc[0]
    
    # Get all reviews for these place_ids
    sub_reviews = reviews[reviews["place_id"].isin(place_ids)]
    
    if sub_reviews.empty:
        results.append({
            "title": title_original,
            "place_ids": place_ids,
            "has_reviews": False,
            "review_overlap": "N/A (no reviews)"
        })
        continue
    
    # Check total and unique review texts
    total_reviews = len(sub_reviews)
    unique_texts = len(sub_reviews.drop_duplicates(subset=["user", "text", "date"]))
    
    identical = total_reviews == unique_texts
    
    # Determine overlap between the place_ids
    overlap_ratio = unique_texts / total_reviews if total_reviews else 0
    
    results.append({
        "title": title_original,
        "place_ids": place_ids,
        "total_reviews": total_reviews,
        "unique_texts": unique_texts,
        "overlap_ratio": round(overlap_ratio, 3),
        "review_overlap": "identical" if identical else "different"
    })

# Convert to DataFrame
review_overlap_df = pd.DataFrame(results)

# Sort to show the different ones first
review_overlap_df = review_overlap_df.sort_values(by="review_overlap", ascending=True).reset_index(drop=True)

# Display
print(f"Total restaurants checked: {len(review_overlap_df)}")
review_overlap_df.head(20)

Total restaurants checked: 186


,title,place_ids,total_reviews,unique_texts,overlap_ratio,review_overlap
0,5th Street,"[Busy8uaGSr4xVTDK64KCcg, 5th-street-christchurch]",6,6,1.0,identical
1,Pies and Coffee,"[IO-ZvolUGtPAJScNnRDGrA, pies-and-coffee-chris...",1,1,1.0,identical
2,Portershed,"[o7caz3L5WXwMpwe2xKVoUg, portershed-christchurch]",3,3,1.0,identical
3,Pot Sticker Dumpling Bar,"[Do7ayiKlaaYwppPZ3CRYWw, pot-sticker-dumpling-...",12,12,1.0,identical
4,Prince of Persia,"[xY8ObYwNWUGknGmZNSvBXQ, prince-of-persia-chri...",1,1,1.0,identical
5,Pure Cafe Co.,"[pg1DJsdU9vSp6WLJcxRxAw, pure-cafe-co-christch...",1,1,1.0,identical
6,Rangoon Ruby,"[nCARbbvdnfXT82QCTE4igA, rangoon-ruby-christch...",5,5,1.0,identical
7,Raw Sugar Cafe,"[BvIxqkZtptUFvW1TmupRIQ, raw-sugar-cafe-christ...",1,1,1.0,identical
8,Reality Bites,"[ZMIUHjkp5PyG2JumFrd7yw, reality-bites-christc...",1,1,1.0,identical
9,Red Elephant,"[9IH4d4GfvmlDL1LpazUt7Q, red-elephant-christch...",1,1,1.0,identical


In [54]:
review_overlap_df['review_overlap'].value_counts()

review_overlap
identical    186
Name: count, dtype: int64

### HANDLE DUPLICATE IN PLACE_ID PER RESTAURANT

In [55]:
restaurant_id.head()

,place_id,title,rating,reviews,price,categories,link
0,wEYj3B_EH6JS6DbfmZQudw,Twenty Seven Steps,4.8,48,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
1,twenty-seven-steps-christchurch,Twenty Seven Steps,4.8,48,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
2,5ZOai8-HCajxJ4gnjqsAdw,OGB,4.9,9,NaN,"Bars, New Zealand",https://www.yelp.com/biz/ogb-christchurch?osq=...
3,ogb-christchurch,OGB,4.9,9,NaN,"Bars, New Zealand",https://www.yelp.com/biz/ogb-christchurch?osq=...
4,iL_451H-5M9tmaAsFqKZrw,Orange House,5.0,2,NaN,Chinese,https://www.yelp.com/biz/orange-house-christch...


In [80]:
# Find edge-case restaurants (≥3 place_ids and mix of hash/slug)
import pandas as pd

# Start from your loaded DataFrame
df = restaurant_id.copy()
df["title_lower"] = df["title"].str.lower().str.strip()

# Heuristic: Yelp hash IDs have NO hyphen and at least one uppercase char
def is_hash(pid: str) -> bool:
    s = str(pid)
    return ("-" not in s) and any(c.isupper() for c in s)

df["is_hash"] = df["place_id"].apply(is_hash)
df["is_slug"] = ~df["is_hash"]

# Group stats per restaurant
grp = (
    df.groupby("title_lower")
      .agg(
          num_ids=("place_id", "nunique"),
          num_hash=("is_hash", "sum"),
          num_slug=("is_slug", "sum")
      )
      .reset_index()
)

# Edge case definition: at least 3 ids AND both hash & slug present
edge_titles = grp.query("num_ids >= 3 and num_hash >= 1 and num_slug >= 1").copy()

print(f"Edge-case restaurants found: {len(edge_titles)}")
edge_titles.head(10)

Edge-case restaurants found: 3


,title_lower,num_ids,num_hash,num_slug
53,fox & ferret public ale house,4,2,2
61,hachi hachi,6,3,3
92,mcdonald’s,4,2,2


In [75]:
# Inspect all rows for those edge-case titles
edge_rows = (
    df[df["title_lower"].isin(edge_titles["title_lower"])]
    .sort_values(["title_lower"])  # keeps original row order within each title
    .reset_index(drop=True)
)

edge_rows[["title", "place_id", "is_hash", "rating", "reviews", "categories"]].head(30)

,title,place_id,is_hash,rating,reviews,categories
0,Fox & Ferret Public Ale House,o0io1bT7fN4rsam461_j_g,True,4.0,1,New Zealand
1,Fox & Ferret Public Ale House,fox-and-ferret-public-ale-house-christchurch,False,4.0,1,New Zealand
2,Fox & Ferret Public Ale House,LceHmZ1Z2pCl1WrVnVNptA,True,2.6,8,New Zealand
3,Fox & Ferret Public Ale House,fox-and-ferret-public-ale-house-christchurch-2,False,2.6,8,New Zealand
4,Hachi Hachi,MWnLwbV6tyBEj1LCd15lAQ,True,3.8,11,Japanese
5,Hachi Hachi,hachi-hachi-christchurch-2,False,3.8,11,Japanese
6,Hachi Hachi,W3bRohV8kGbiTAWLMu9SUw,True,3.2,5,"Japanese, Sushi Bars, Noodles"
7,Hachi Hachi,hachi-hachi-christchurch-3,False,3.2,5,"Japanese, Sushi Bars, Noodles"
8,Hachi Hachi,dwebhjgrd7u1pjADqZH2RA,True,2.0,1,Japanese
9,Hachi Hachi,hachi-hachi-christchurch,False,2.0,1,Japanese


In [81]:
# Keep only hash IDs for these edge-case restaurants
clean_edge = df[
    ~((df["title_lower"].isin(edge_titles["title_lower"])) & (df["is_slug"]))
].reset_index(drop=True)

print(f"Original rows: {len(df)}")
print(f"After removing slugs (only for ≥3-ID mixed titles): {len(clean_edge)}")

Original rows: 382
After removing slugs (only for ≥3-ID mixed titles): 375


- remove duplicate place_ids per restaurant title (keeping the first),

- but skip the 3 special cases:
- "Fox & Ferret Public Ale House", "Hachi Hachi", and "McDonald’s" —
- because these have multiple hash IDs with different review counts that you’ll manually inspect later.

In [86]:
df_cleaned_edge = clean_edge

# Define your edge-case titles
edge_case_titles = ["Fox & Ferret Public Ale House", "Hachi Hachi", "McDonald’s"]

# Normalize title for consistency
df_cleaned_edge["title_lower"] = df_cleaned_edge["title"].str.lower().str.strip()

# Filter out the edge cases for now
non_edge = df_cleaned_edge[~df_cleaned_edge["title"].isin(edge_case_titles)]

# Drop duplicates among non-edge titles (keep first place_id)
non_edge_unique = non_edge.drop_duplicates(subset="title_lower", keep="first")

# Combine back edge cases (keep all their hashed IDs)
restaurant_final = pd.concat([non_edge_unique, df_cleaned_edge[df_cleaned_edge["title"].isin(edge_case_titles)]], ignore_index=True)

print(f"Original rows: {len(df_cleaned_edge)}")
print(f"After cleaning (keeping all hash IDs for edge cases): {len(restaurant_final)}")

# Preview result
restaurant_final.sort_values("title").head(15)

Original rows: 375
After cleaning (keeping all hash IDs for edge cases): 190


,place_id,title,rating,reviews,price,categories,link,title_lower,is_hash,is_slug
4,Busy8uaGSr4xVTDK64KCcg,5th Street,5.0,6,NaN,"Wine Bars, Tapas/Small Plates, Cocktail Bars",https://www.yelp.com/biz/5th-street-christchur...,5th street,True,False
99,Bonsa2BwNhdqFWVmrn1Rag,Abalone Thai Restaurant,4.0,4,NaN,Thai,https://www.yelp.com/biz/abalone-thai-restaura...,abalone thai restaurant,True,False
75,5aSFsf5QbevEYxH3KTkRKA,Amaterrace Teppanyaki,4.0,2,NaN,Teppanyaki,https://www.yelp.com/biz/amaterrace-teppanyaki...,amaterrace teppanyaki,True,False
21,zHk2a3tQdyldANFOeqjR1A,Amazonita,4.1,10,NaN,Mediterranean,https://www.yelp.com/biz/amazonita-christchurc...,amazonita,True,False
152,FfB040u_JQh6PnbsrLQM_w,Anna’s Cafe,4.0,1,NaN,"Breakfast & Brunch, Cafes",https://www.yelp.com/biz/annas-cafe-christchur...,anna’s cafe,True,False
176,tZzZ6KZpBITfPH1c9YQhNQ,Antigua Boatsheds Canoe Hire,4.7,3,$$,"Coffee & Tea, Cafes",https://www.yelp.com/biz/antigua-boatsheds-can...,antigua boatsheds canoe hire,True,False
170,y4jnOb4jgLMe029wAaABdA,Arbo Cafe,5.0,1,NaN,Cafes,https://www.yelp.com/biz/arbo-cafe-christchurc...,arbo cafe,True,False
103,3SGm_b79hqJRgsWayAukxA,Arjee Bhajee,4.5,2,NaN,Indian,https://www.yelp.com/biz/arjee-bhajee-christch...,arjee bhajee,True,False
76,NbKSaN-diw8E4S2ub2hXsQ,Aroy Thai Restaurant,5.0,2,NaN,Restaurants,https://www.yelp.com/biz/aroy-thai-restaurant-...,aroy thai restaurant,False,True
25,RDYakuJC8b6R_BWPDAGd4g,Bacon Brothers,4.3,23,$,"Breakfast & Brunch, Beer Bar, Burgers",https://www.yelp.com/biz/bacon-brothers-christ...,bacon brothers,True,False


In [87]:
restaurant_final.to_csv("Christchurch_place_ids_cleaned.csv", index=False)
print("Saved final cleaned file: Christchurch_place_ids_cleaned.csv")

Saved final cleaned file: Christchurch_place_ids_cleaned.csv


In [91]:
df_reviews = pd.read_csv("christchurch_reviews_all_pages.csv")

df_reviews.head()

,place_id,review_type,user,rating,date,text,review_position,user_address,useful,cool,funny,user_link
0,wEYj3B_EH6JS6DbfmZQudw,recommended,John B.,5,2025-03-27T05:04:34Z,A warm greeting followed by a delicious dinner...,1,"Falmouth, MA",0,0,0,https://www.yelp.com/user_details?userid=MgHxF...
1,wEYj3B_EH6JS6DbfmZQudw,recommended,Azure G.,5,2025-02-14T04:09:56Z,Twenty Seven Steps - what a spot! Intimate spa...,2,"Denver, CO",0,0,0,https://www.yelp.com/user_details?userid=Z583n...
2,wEYj3B_EH6JS6DbfmZQudw,recommended,Calista C.,5,2025-01-29T21:31:56Z,Visited here while on my stop in Christchurch....,3,"Irvine, CA",2,2,0,https://www.yelp.com/user_details?userid=PqR0K...
3,wEYj3B_EH6JS6DbfmZQudw,recommended,Jasper R.,5,2024-11-20T04:47:31Z,Everything in here was amazing! One of my favo...,4,"Lathrop, CA",0,0,0,https://www.yelp.com/user_details?userid=IaX6k...
4,wEYj3B_EH6JS6DbfmZQudw,recommended,Andy M.,4,2024-10-25T05:19:39Z,"Cool atmosphere, great service, and the good f...",5,"Cincinnati, OH",0,0,0,https://www.yelp.com/user_details?userid=ODxAj...


In [94]:
df_reviews["place_id"].nunique()

190

In [96]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086 entries, 0 to 1085
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   place_id         1086 non-null   object
 1   review_type      1086 non-null   object
 2   user             1086 non-null   object
 3   rating           1086 non-null   int64 
 4   date             1086 non-null   object
 5   text             1086 non-null   object
 6   review_position  1086 non-null   int64 
 7   user_address     1086 non-null   object
 8   useful           1086 non-null   int64 
 9   cool             1086 non-null   int64 
 10  funny            1086 non-null   int64 
 11  user_link        1086 non-null   object
dtypes: int64(5), object(7)
memory usage: 101.9+ KB


In [97]:
restaurant_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   place_id     190 non-null    object 
 1   title        190 non-null    object 
 2   rating       190 non-null    float64
 3   reviews      190 non-null    int64  
 4   price        65 non-null     object 
 5   categories   190 non-null    object 
 6   link         190 non-null    object 
 7   title_lower  190 non-null    object 
 8   is_hash      190 non-null    bool   
 9   is_slug      190 non-null    bool   
dtypes: bool(2), float64(1), int64(1), object(6)
memory usage: 12.4+ KB


In [98]:
# Join 2 columns 
# Select only the needed columns from restaurant_final
restaurant_info = restaurant_final[["place_id", "title", "price", "categories", "link"]]

# Merge on place_id (left join to keep all reviews)
chc_reviews_df = df_reviews.merge(restaurant_info, on="place_id", how="left")

# Preview result
print(f"Original reviews: {len(df_reviews)}")
print(f"Merged reviews: {len(chc_reviews_df)}")
chc_reviews_df.head()

Original reviews: 1086
Merged reviews: 1086


,place_id,review_type,user,rating,date,text,review_position,user_address,useful,cool,funny,user_link,title,price,categories,link
0,wEYj3B_EH6JS6DbfmZQudw,recommended,John B.,5,2025-03-27T05:04:34Z,A warm greeting followed by a delicious dinner...,1,"Falmouth, MA",0,0,0,https://www.yelp.com/user_details?userid=MgHxF...,Twenty Seven Steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
1,wEYj3B_EH6JS6DbfmZQudw,recommended,Azure G.,5,2025-02-14T04:09:56Z,Twenty Seven Steps - what a spot! Intimate spa...,2,"Denver, CO",0,0,0,https://www.yelp.com/user_details?userid=Z583n...,Twenty Seven Steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
2,wEYj3B_EH6JS6DbfmZQudw,recommended,Calista C.,5,2025-01-29T21:31:56Z,Visited here while on my stop in Christchurch....,3,"Irvine, CA",2,2,0,https://www.yelp.com/user_details?userid=PqR0K...,Twenty Seven Steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
3,wEYj3B_EH6JS6DbfmZQudw,recommended,Jasper R.,5,2024-11-20T04:47:31Z,Everything in here was amazing! One of my favo...,4,"Lathrop, CA",0,0,0,https://www.yelp.com/user_details?userid=IaX6k...,Twenty Seven Steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
4,wEYj3B_EH6JS6DbfmZQudw,recommended,Andy M.,4,2024-10-25T05:19:39Z,"Cool atmosphere, great service, and the good f...",5,"Cincinnati, OH",0,0,0,https://www.yelp.com/user_details?userid=ODxAj...,Twenty Seven Steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...


In [111]:
chc_reviews_df["date"] = pd.to_datetime(chc_reviews_df["date"], errors="coerce")
chc_reviews_df["title"] = chc_reviews_df["title"].str.lower().str.strip()

In [112]:
chc_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086 entries, 0 to 1085
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   place_id         1086 non-null   object        
 1   review_type      1086 non-null   object        
 2   user             1086 non-null   object        
 3   rating           1086 non-null   int64         
 4   date             1086 non-null   datetime64[ns]
 5   text             1086 non-null   object        
 6   review_position  1086 non-null   int64         
 7   user_address     1086 non-null   object        
 8   useful           1086 non-null   int64         
 9   cool             1086 non-null   int64         
 10  funny            1086 non-null   int64         
 11  user_link        1086 non-null   object        
 12  title            1086 non-null   object        
 13  price            710 non-null    object        
 14  categories       1086 non-null   object 

In [115]:
chc_reviews_df.head()

,place_id,review_type,user,rating,date,text,review_position,user_address,useful,cool,funny,user_link,title,price,categories,link
0,wEYj3B_EH6JS6DbfmZQudw,recommended,John B.,5,2025-03-27 05:04:34,A warm greeting followed by a delicious dinner...,1,"Falmouth, MA",0,0,0,https://www.yelp.com/user_details?userid=MgHxF...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
1,wEYj3B_EH6JS6DbfmZQudw,recommended,Azure G.,5,2025-02-14 04:09:56,Twenty Seven Steps - what a spot! Intimate spa...,2,"Denver, CO",0,0,0,https://www.yelp.com/user_details?userid=Z583n...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
2,wEYj3B_EH6JS6DbfmZQudw,recommended,Calista C.,5,2025-01-29 21:31:56,Visited here while on my stop in Christchurch....,3,"Irvine, CA",2,2,0,https://www.yelp.com/user_details?userid=PqR0K...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
3,wEYj3B_EH6JS6DbfmZQudw,recommended,Jasper R.,5,2024-11-20 04:47:31,Everything in here was amazing! One of my favo...,4,"Lathrop, CA",0,0,0,https://www.yelp.com/user_details?userid=IaX6k...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
4,wEYj3B_EH6JS6DbfmZQudw,recommended,Andy M.,4,2024-10-25 05:19:39,"Cool atmosphere, great service, and the good f...",5,"Cincinnati, OH",0,0,0,https://www.yelp.com/user_details?userid=ODxAj...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...


In [117]:
# Find duplicate place_id with same title
# Count how many unique place_id per title
title_placeid_counts = (
    chc_reviews_df.groupby("title")["place_id"]
    .nunique()
    .reset_index(name="unique_place_ids")
)

# Filter where the same title has more than one place_id
titles_with_multiple_ids = title_placeid_counts[title_placeid_counts["unique_place_ids"] > 1]

print(f"Total titles with multiple place_ids: {len(titles_with_multiple_ids)}")

# Show a few examples
titles_with_multiple_ids.head(20)

Total titles with multiple place_ids: 3


,title,unique_place_ids
53,fox & ferret public ale house,2
61,hachi hachi,3
92,mcdonald’s,2


In [118]:
# Get all rows for titles that have more than one unique place_id
duplicate_placeid_rows = (
    chc_reviews_df[chc_reviews_df["title"].isin(titles_with_multiple_ids["title"])]
    .sort_values(["title", "place_id"])
    .reset_index(drop=True)
)

print(f"Total rows from titles with multiple place_ids: {len(duplicate_placeid_rows)}")

# Display a sample
duplicate_placeid_rows[["title", "place_id", "rating", "date", "text"]].head(20)

Total rows from titles with multiple place_ids: 29


,title,place_id,rating,date,text
0,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,1,2018-02-01 23:55:41,"Wouldn't serve me a beer even though I'm 27, 6..."
1,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,4,2017-01-08 00:15:42,Had a great meal with my family here. Staff wa...
2,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,5,2016-07-25 05:06:23,"We loved it! Great atmosphere, live music, gre..."
3,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,1,2015-12-19 23:31:51,Went in at 5pm for a pint OF COKE(!) and was I...
4,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,2,2015-10-29 23:32:22,The Halloween themed party was very fitting la...
5,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,2,2015-10-10 00:15:14,Its a nice pub with a good vibe. Its good for ...
6,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,4,2014-09-26 23:27:57,Reviewing on visiting during a pre lunch week ...
7,fox & ferret public ale house,LceHmZ1Z2pCl1WrVnVNptA,2,2014-09-14 10:03:12,I find the only good time to go drinking in th...
8,fox & ferret public ale house,o0io1bT7fN4rsam461_j_g,4,2015-06-29 02:41:56,This is a pretty great bar located in The Palm...
9,hachi hachi,MWnLwbV6tyBEj1LCd15lAQ,4,2025-03-06 01:44:58,My husband and I had lunch here. I was craving...


### HANDLE DUPLICATE PLACE_ID - SAME TITLE - DIFFERENT REVIEWS
- we will convert it to keep the most recent place_id only 

In [119]:
# Step 2: Find the most recent place_id per restaurant title
latest_per_place = (
    chc_reviews_df.groupby(["title", "place_id"])["date"]
    .max()
    .reset_index()
)

In [120]:
# Step 3: For each title, get the place_id with the newest date
newest_place_ids = (
    latest_per_place.sort_values(["title", "date"], ascending=[True, False])
    .drop_duplicates(subset=["title"], keep="first")
    .rename(columns={"place_id": "canonical_place_id"})
    [["title", "canonical_place_id"]]
)

In [121]:
# Step 4: Merge back — map all old place_ids to the canonical one
chc_reviews_unified = chc_reviews_df.merge(
    newest_place_ids, on="title", how="left"
)

In [122]:
# Step 5: Replace place_id with canonical id
chc_reviews_unified["place_id"] = chc_reviews_unified["canonical_place_id"]
chc_reviews_unified = chc_reviews_unified.drop(columns=["canonical_place_id"])

print(f"All reviews unified. Total reviews: {len(chc_reviews_unified)}")
print(f"Unique place_ids before: {chc_reviews_df['place_id'].nunique()}")
print(f"Unique place_ids after: {chc_reviews_unified['place_id'].nunique()}")

All reviews unified. Total reviews: 1086
Unique place_ids before: 190
Unique place_ids after: 186


In [123]:
chc_reviews_unified[["title", "place_id", "rating", "date"]].head(10)

,title,place_id,rating,date
0,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2025-03-27 05:04:34
1,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2025-02-14 04:09:56
2,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2025-01-29 21:31:56
3,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2024-11-20 04:47:31
4,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,4,2024-10-25 05:19:39
5,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2024-07-21 06:25:26
6,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2024-05-15 09:22:11
7,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,4,2024-04-12 06:52:26
8,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2024-04-06 05:38:35
9,twenty seven steps,wEYj3B_EH6JS6DbfmZQudw,5,2023-12-22 00:21:34


In [125]:
chc_reviews_unified['place_id'].nunique()

186

In [126]:
chc_reviews_unified['title'].nunique()

186

In [130]:
# check the edge cases
target_titles = ["hachi hachi", "mcdonald’s", "fox & ferret public ale house"]

# Filter only those titles
selected_restaurants = (
    chc_reviews_unified[chc_reviews_unified["title"].isin(target_titles)]
    .groupby("title")["place_id"]
    .unique()
    .reset_index()
)

print("Canonical place_id(s) for selected restaurants:")
selected_restaurants

Canonical place_id(s) for selected restaurants:


,title,place_id
0,fox & ferret public ale house,[LceHmZ1Z2pCl1WrVnVNptA]
1,hachi hachi,[MWnLwbV6tyBEj1LCd15lAQ]
2,mcdonald’s,[8UcIcdTWPSIEyMOAcUbXEw]


In [133]:
chc_reviews_df = chc_reviews_unified

In [135]:
chc_reviews_df.head()

,place_id,review_type,user,rating,date,text,review_position,user_address,useful,cool,funny,user_link,title,price,categories,link
0,wEYj3B_EH6JS6DbfmZQudw,recommended,John B.,5,2025-03-27 05:04:34,A warm greeting followed by a delicious dinner...,1,"Falmouth, MA",0,0,0,https://www.yelp.com/user_details?userid=MgHxF...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
1,wEYj3B_EH6JS6DbfmZQudw,recommended,Azure G.,5,2025-02-14 04:09:56,Twenty Seven Steps - what a spot! Intimate spa...,2,"Denver, CO",0,0,0,https://www.yelp.com/user_details?userid=Z583n...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
2,wEYj3B_EH6JS6DbfmZQudw,recommended,Calista C.,5,2025-01-29 21:31:56,Visited here while on my stop in Christchurch....,3,"Irvine, CA",2,2,0,https://www.yelp.com/user_details?userid=PqR0K...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
3,wEYj3B_EH6JS6DbfmZQudw,recommended,Jasper R.,5,2024-11-20 04:47:31,Everything in here was amazing! One of my favo...,4,"Lathrop, CA",0,0,0,https://www.yelp.com/user_details?userid=IaX6k...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...
4,wEYj3B_EH6JS6DbfmZQudw,recommended,Andy M.,4,2024-10-25 05:19:39,"Cool atmosphere, great service, and the good f...",5,"Cincinnati, OH",0,0,0,https://www.yelp.com/user_details?userid=ODxAj...,twenty seven steps,$$,Modern European,https://www.yelp.com/biz/twenty-seven-steps-ch...


In [134]:
chc_reviews_df.to_csv("christchurch-yelp-reviews.csv", index=False)
print("Saved final cleaned file: christchurch-yelp-reviews.csv")

Saved final cleaned file: christchurch-yelp-reviews.csv


In [145]:
# Save as Parquet
chc_reviews_df.to_parquet("data/chc-yelp-reviews-v2/christchurch-yelp-reviews.parquet", index=False)
print("Saved final cleaned file: christchurch-yelp-reviews.parquet")

Saved final cleaned file: christchurch-yelp-reviews.parquet


In [147]:
import json

# Ensure 'date' is a string
chc_reviews_df["date"] = chc_reviews_df["date"].astype(str)

# Group all reviews by restaurant
grouped_data = []

for title, group in chc_reviews_df.groupby("title"):
    record = {
        "restaurant": title,
        "place_id": group["place_id"].iloc[0],
        "reviews": group[[
            "user", "rating", "date", "text", "review_type",
            "user_address", "useful", "cool", "funny"
        ]].to_dict(orient="records")
    }
    grouped_data.append(record)

# Save as pretty JSON
with open("chc-yelp-reviews.json", "w", encoding="utf-8") as f:
    json.dump(grouped_data, f, ensure_ascii=False, indent=4)

print("Exported grouped JSON to 'chc-yelp-reviews.json'")

Exported grouped JSON to 'chc-yelp-reviews.json'


In [148]:
with open("chc-yelp-reviews.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Print total restaurants and first few entries
print(f"Total restaurants: {len(data)}\n")
print(json.dumps(data[:2], indent=4, ensure_ascii=False))  

Total restaurants: 186

[
    {
        "restaurant": "5th street",
        "place_id": "Busy8uaGSr4xVTDK64KCcg",
        "reviews": [
            {
                "user": "Andy M.",
                "rating": 5,
                "date": "2024-10-24 07:32:01",
                "text": "One of the finest farm to table restaurants I've ever been to in the world. The head chef is passionate about his craft, great staff, service and ambience. Don't miss this gem!\nUpdate. So good I came back 2 days later !",
                "review_type": "recommended",
                "user_address": "Cincinnati, OH",
                "useful": 0,
                "cool": 0,
                "funny": 0
            },
            {
                "user": "Mark C.",
                "rating": 5,
                "date": "2024-04-19 20:19:21",
                "text": "What a treat in Christchurch. Off the beaten path. In an  industrial area. But inside you feel like you're in a tropical garden.\nThe food was excel